In [131]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

team_data = pd.read_csv("../Stats_competition-/final_opponent_and_team_data.csv")

In [132]:
threshold = 6

In [133]:
team_data

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,...,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d
0,2024-11-04,Duke,Maine,H,0.1,125.3,95.2,130.6,64.3,17.7,...,39.5,19.0,19.4,31.6,18-43,3-14,62,96,108.9,110.8
1,2024-11-08,Duke,Army,H,0.1,124.7,92.3,141.0,61.3,11.3,...,39.1,18.3,23.3,14.1,13-35,8-29,58,100,107.2,119.0
2,2024-11-12,Duke,Kentucky,N,-0.2,106.8,86.4,95.7,42.3,9.3,...,47.6,14.6,23.3,38.1,15-38,10-25,77,72,119.6,88.1
3,2024-11-16,Duke,Wofford,H,-0.1,124.7,56.9,133.7,61.3,15.5,...,28.9,29.5,29.3,5.3,9-24,5-33,35,86,69.8,115.5
4,2024-11-22,Duke,Arizona,A,0.6,111.7,75.9,101.9,50.0,20.7,...,45.3,22.2,16.7,20.8,15-30,6-23,55,69,98.9,90.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2024-11-27,Michigan,Xavier,N,0.3,110.9,73.9,109.9,58.2,14.1,...,41.4,26.8,25.6,15.5,12-35,8-23,53,78,85.8,96.0
298,2024-12-03,Michigan,Wisconsin,A,1.0,102.7,84.4,99.3,44.1,17.8,...,39.1,14.8,34.8,29.7,16-37,6-27,64,67,103.9,89.6
299,2024-12-07,Michigan,Iowa,H,1.4,121.1,99.5,114.5,58.6,22.9,...,47.4,5.4,30.6,22.1,23-50,9-27,83,85,120.5,104.8
300,2024-12-10,Michigan,Arkansas,N,0.9,123.6,108.4,112.3,64.4,21.9,...,56.8,18.1,33.3,33.3,24-43,9-23,89,87,124.8,102.1


In [134]:
team_data["Location"] = np.where(
    team_data["Location"] == "N", 0, np.where(team_data["Location"] == "H", 1, -1)
)

In [135]:
columns_to_convert = [
    "Location",
    "ADJO",
    "ADJD",
    "EFG%",
    "TO%",
    "OR%",
    "FTR",
    "Opp EFG%",
    "Opp TO%",
    "Opp OR%",
    "Opp FTR",
]
for col in columns_to_convert:
    team_data[col] = pd.to_numeric(team_data[col], errors="coerce")

In [136]:
team_data_cleaned = team_data.dropna()

X = team_data_cleaned[columns_to_convert]
y_team = team_data_cleaned["Team_score"]
y_opp = team_data_cleaned["Opponent_score"]

X_train, X_test, y_team_train, y_team_test = train_test_split(
    X, y_team, test_size=0.2, random_state=42
)
y_opp_train, y_opp_test = train_test_split(y_opp, test_size=0.2, random_state=42)

model_team = LinearRegression()
model_team.fit(X_train, y_team_train)
team_pred = model_team.predict(X_test)

model_opp = LinearRegression()
model_opp.fit(X_train, y_opp_train)
opp_pred = model_opp.predict(X_test)

team_rmse = mean_squared_error(y_team_test, team_pred, squared=False)
opp_rmse = mean_squared_error(y_opp_test, opp_pred, squared=False)
print(f"Team Score RMSE: {team_rmse}, Opponent Score RMSE: {opp_rmse}")

Team Score RMSE: 5.694257875441654, Opponent Score RMSE: 5.397831922401339


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [137]:
team_accuracy = (abs(team_pred - y_team_test) <= threshold).mean() * 100
opp_accuracy = (abs(opp_pred - y_opp_test) <= threshold).mean() * 100
print(f"Team Score Accuracy: {team_accuracy:.2f}%")
print(f"Opponent Score Accuracy: {opp_accuracy:.2f}%")

Team Score Accuracy: 68.85%
Opponent Score Accuracy: 73.77%


In [138]:
predict_games = pd.read_csv("../Stats_competition-/basketball_games_data.csv")

In [139]:
predict_games["Location"] = np.where(
    predict_games["Location"] == "Neutral",
    0,
    np.where(predict_games["Location"] == "Home", 1, -1),
)

In [140]:
predict_games

,date,Location,Team,Opponent,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,opp_adj_o,opp_adj_d
0,2024-12-23,1,North Dakota St.,Cal St. Bakersfield,133.4,89.4,62.7,8.8,20.0,68.6,46.3,16.1,21.2,38.9,85.2,125.3
1,2024-12-23,0,Charleston,Loyola Chicago,102.1,91.8,56.0,20.9,9.7,27.6,46.6,24.8,31.6,34.5,95.6,97.9
2,2024-12-23,0,Charlotte,Murray St.,118.6,110.4,50.0,17.4,36.4,60.0,46.4,8.7,28.2,52.2,106.0,115.3
3,2024-12-23,1,Tennessee,Middle Tennessee,112.6,95.4,57.1,15.7,25.8,41.1,48.3,15.7,15.8,23.7,116.3,98.5
4,2024-12-23,-1,Nebraska,Hawaii,107.5,93.1,53.6,19.3,33.3,20.0,47.9,25.7,32.1,31.9,103.3,103.5
5,2024-12-23,0,Oregon St.,Oakland,110.6,112.1,50.8,16.1,46.2,28.8,50.9,20.5,38.2,42.9,120.8,109.1
6,2024-12-23,-1,Seattle,Washington,120.1,92.5,44.6,11.1,21.9,57.1,47.4,16.7,21.6,40.4,91.8,112.2


In [141]:
X = predict_games[columns_to_convert]

In [142]:
X

,Location,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR
0,1,133.4,89.4,62.7,8.8,20.0,68.6,46.3,16.1,21.2,38.9
1,0,102.1,91.8,56.0,20.9,9.7,27.6,46.6,24.8,31.6,34.5
2,0,118.6,110.4,50.0,17.4,36.4,60.0,46.4,8.7,28.2,52.2
3,1,112.6,95.4,57.1,15.7,25.8,41.1,48.3,15.7,15.8,23.7
4,-1,107.5,93.1,53.6,19.3,33.3,20.0,47.9,25.7,32.1,31.9
5,0,110.6,112.1,50.8,16.1,46.2,28.8,50.9,20.5,38.2,42.9
6,-1,120.1,92.5,44.6,11.1,21.9,57.1,47.4,16.7,21.6,40.4


In [143]:
from pygam import LinearGAM, s, f
from sklearn.metrics import mean_squared_error

# Fit PyGAM for Team Score
gam_team = LinearGAM(
    s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10)
).fit(X_train, y_team_train)
team_gam_pred = gam_team.predict(X_test)

# Fit PyGAM for Opponent Score
gam_opp = LinearGAM(
    s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10)
).fit(X_train, y_opp_train)
opp_gam_pred = gam_opp.predict(X_test)

# Calculate RMSE for PyGAM
team_gam_rmse = mean_squared_error(y_team_test, team_gam_pred, squared=False)
opp_gam_rmse = mean_squared_error(y_opp_test, opp_gam_pred, squared=False)
print(f"Team Score RMSE (PyGAM): {team_gam_rmse}")
print(f"Opponent Score RMSE (PyGAM): {opp_gam_rmse}")

# Calculate accuracy for PyGAM
team_gam_accuracy = (abs(team_gam_pred - y_team_test) <= threshold).mean() * 100
opp_gam_accuracy = (abs(opp_gam_pred - y_opp_test) <= threshold).mean() * 100
print(f"Team Score Accuracy (PyGAM): {team_gam_accuracy:.2f}%")
print(f"Opponent Score Accuracy (PyGAM): {opp_gam_accuracy:.2f}%")

# Predict scores using PyGAM
team_gam_pred_new = gam_team.predict(X)
opp_gam_pred_new = gam_opp.predict(X)

# Add PyGAM predictions to the table
predictions_gam = pd.DataFrame(
    {
        "Predicted Team Score GAM": team_gam_pred_new,
        "Predicted Opponent Score GAM": opp_gam_pred_new,
    }
)

predict_games = pd.concat([predict_games, predictions_gam], axis=1)

Team Score RMSE (PyGAM): 6.986779995501826
Opponent Score RMSE (PyGAM): 6.7824075879007
Team Score Accuracy (PyGAM): 65.57%
Opponent Score Accuracy (PyGAM): 68.85%


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [144]:
team_pred_new = model_team.predict(X)
opp_pred_new = model_opp.predict(X)

predictions = pd.DataFrame(
    {
        "Predicted Team Score LR": team_pred_new,
        "Predicted Opponent Score LR": opp_pred_new,
    }
)

In [145]:
predict_games = pd.concat([predict_games, predictions], axis=1)

In [146]:
predict_games[
    [
        "Team",
        "Opponent",
        "Predicted Team Score LR",
        "Predicted Opponent Score LR",
    ]
]

,Team,Opponent,Predicted Team Score LR,Predicted Opponent Score LR
0,North Dakota St.,Cal St. Bakersfield,99.005598,71.310070
1,Charleston,Loyola Chicago,64.376385,63.129225
2,Charlotte,Murray St.,81.774269,80.570754
3,Tennessee,Middle Tennessee,82.487142,66.657639
4,Nebraska,Hawaii,72.296803,63.402850
5,Oregon St.,Oakland,79.490891,75.211160
6,Seattle,Washington,76.451057,71.630756


In [147]:
# Add columns for averages
predict_games["Average Predicted Team Score"] = predict_games[
    ["Predicted Team Score LR", "Predicted Team Score GAM"]
].mean(axis=1)

predict_games["Average Predicted Opponent Score"] = predict_games[
    ["Predicted Opponent Score LR", "Predicted Opponent Score GAM"]
].mean(axis=1)

# Select columns to display
columns_to_display = [
    "Team",
    "Opponent",
    "Predicted Team Score LR",
    "Predicted Opponent Score LR",
    "Predicted Team Score GAM",
    "Predicted Opponent Score GAM",
    "Average Predicted Team Score",
    "Average Predicted Opponent Score",
]

# Display the updated table
# print(predict_games[columns_to_display].to_string(index=False))
display(predict_games[columns_to_display])

,Team,Opponent,Predicted Team Score LR,Predicted Opponent Score LR,Predicted Team Score GAM,Predicted Opponent Score GAM,Average Predicted Team Score,Average Predicted Opponent Score
0,North Dakota St.,Cal St. Bakersfield,99.005598,71.310070,98.369701,68.199230,98.687649,69.754650
1,Charleston,Loyola Chicago,64.376385,63.129225,70.288591,65.934352,67.332488,64.531789
2,Charlotte,Murray St.,81.774269,80.570754,76.650592,74.620033,79.212430,77.595393
3,Tennessee,Middle Tennessee,82.487142,66.657639,82.440742,65.781922,82.463942,66.219781
4,Nebraska,Hawaii,72.296803,63.402850,75.328404,67.376555,73.812604,65.389702
5,Oregon St.,Oakland,79.490891,75.211160,74.914191,75.305861,77.202541,75.258511
6,Seattle,Washington,76.451057,71.630756,77.425490,70.332972,76.938273,70.981864
